<a href="https://colab.research.google.com/github/kutikova2016/see-me/blob/master/K_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#написал на этой неделе лично Кутиков Иван
#по шаблону с курса "Компьютерное зрение" от Samsung
#С API  Kaggle разбирался самостоятельно
#Score 0.95742
!pip install kaggle --upgrade
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
api_token = {"username":"nontar","key":"e2a8107e332106824a40ef9ba54c1f31"}
import json
with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v
!kaggle competitions download -c digit-recognizer

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)
mkdir: cannot create directory ‘/root/.kaggle’: File exists
Kaggle API 1.5.4
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
import numpy as np
import pandas as pd
import torch
import random
import matplotlib.pyplot as plt
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [0]:
test = pd.read_csv('test.csv.zip', compression='zip')
train = pd.read_csv('train.csv.zip', compression='zip')
y_train = torch.from_numpy(np.array(train['label']))
torch_train = torch.Tensor(len(train), 1, 28, 28)
for i in range(len(train)):
    st = train.iloc[i]
    N = np.array(st[1:])
    N = N.reshape((28, 28))
    T = torch.from_numpy(N)
    torch_train[i][0] = T
L = len(torch_train)
assert len(y_train) == L, L
L = int(L*0.9)
X_train, X_valid = torch_train[:L], torch_train[L:]
y_train, y_valid = y_train[:L], y_train[L:]

In [0]:
class LeNet5(torch.nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv0 = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)
        self.act0  = torch.nn.Tanh()
        self.pool0 = torch.nn.MaxPool2d(kernel_size=2, stride=1)

        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=20, kernel_size=3, padding=1)
        self.act1  = torch.nn.Tanh()
        self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = torch.nn.Conv2d(in_channels=20, out_channels=16, kernel_size=5, padding=0)
        self.act2  = torch.nn.Tanh()
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc3   = torch.nn.Linear(4*4*16, 800)
        self.act3  = torch.nn.Tanh()
        self.drop3 = torch.nn.Dropout(p=0.8)

        self.fc4   = torch.nn.Linear(800, 400)
        self.act4  = torch.nn.ReLU()
        self.drop4 = torch.nn.Dropout(p=0.8)

        self.fc5   = torch.nn.Linear(400, 400)
        self.act5  = torch.nn.Tanh()
        self.drop5 = torch.nn.Dropout(p=0.8)

        self.fc6   = torch.nn.Linear(400, 10)

    def forward(self, x):
        x = self.conv0(x)
        x = self.act0(x)
        x = self.pool0(x)
        

        x = self.conv1(x)
        
        x = self.act1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

        x = self.fc3(x)
        x = self.drop3(x)
        x = self.act3(x)

        x = self.fc4(x)
        x = self.drop4(x)
        x = self.act4(x)
        
        #x = self.fc5(x)
        #x = self.drop5(x)
        #x = self.act5(x)

        x = self.fc6(x)
        return x
    def predict(self, x):
        x = self.conv0(x)
        x = self.act0(x)
        x = self.pool0(x)

        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)
        
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))

        
        x = self.fc3(x)
        x = self.act3(x)
        

        x = self.fc4(x)
        x = self.act4(x)
        
        #x = self.fc5(x)
        #x = self.act5(x)

        x = self.fc6(x)
        return x
lenet5 = LeNet5()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
lenet5 = lenet5.to(device)

loss = torch.nn.CrossEntropyLoss()


In [0]:
test_ERR_history = []
test_loss_history = []

In [0]:
optimizer = torch.optim.Adam(lenet5.parameters(), lr=0.0001)
batch_size = 800
X_valid = X_valid.to(device)
y_valid = y_valid.to(device)

for epoch in range(100000):
    order = np.random.permutation(len(X_train))
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()

        batch_indexes = order[start_index:start_index+batch_size]
        X_batch = X_train[batch_indexes].to(device)
        y_batch = y_train[batch_indexes].to(device)

        for i in range(len(X_batch)):
            for k in range(100):
                X_batch[i, 0, random.randint(0, 27), random.randint(0, 27)] = random.randint(0, 255)
        preds = lenet5.forward(X_batch)

        loss_value = loss(preds, y_batch)
        loss_value.backward()

        optimizer.step()
        #####
        test_preds = lenet5.predict(X_valid)
        test_loss_history.append(loss(test_preds, y_valid).data.cpu())
        ERR = 1-(test_preds.argmax(dim=1) == y_valid).float().mean().data.cpu()
        test_ERR_history.append(ERR)

        print(ERR)


tensor(0.0429)
tensor(0.0429)
tensor(0.0440)
tensor(0.0443)
tensor(0.0450)
tensor(0.0452)
tensor(0.0455)
tensor(0.0457)
tensor(0.0460)
tensor(0.0455)
tensor(0.0452)
tensor(0.0450)
tensor(0.0445)
tensor(0.0443)
tensor(0.0448)
tensor(0.0448)
tensor(0.0448)
tensor(0.0452)
tensor(0.0457)
tensor(0.0455)
tensor(0.0452)
tensor(0.0450)
tensor(0.0455)
tensor(0.0460)
tensor(0.0460)
tensor(0.0462)
tensor(0.0462)
tensor(0.0460)
tensor(0.0460)
tensor(0.0455)
tensor(0.0450)
tensor(0.0445)
tensor(0.0438)
tensor(0.0438)
tensor(0.0438)
tensor(0.0440)
tensor(0.0438)
tensor(0.0438)
tensor(0.0438)
tensor(0.0429)
tensor(0.0424)
tensor(0.0424)
tensor(0.0414)
tensor(0.0417)
tensor(0.0424)
tensor(0.0426)
tensor(0.0426)
tensor(0.0431)


KeyboardInterrupt: ignored

In [0]:
X_test = torch.Tensor(len(test), 1, 28, 28)
for i in range(len(test)):
    st = test.iloc[i]
    N = np.array(st)
    N = N.reshape((28, 28))
    T = torch.from_numpy(N)
    X_test[i][0] = T

In [0]:
y_test = lenet5.predict(X_test).argmax(dim=1)
y_test

tensor([2, 0, 9,  ..., 3, 9, 2])

In [0]:
y_test = pd.DataFrame(y_test).reset_index()
y_test.rename(columns={'index': 'ImageId', 0: 'Label'}, inplace=True)
y_test['ImageId'] = y_test['ImageId'] + 1
y_test

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9


In [0]:
y_test.to_csv("submission.csv", index=False)
!kaggle competitions submit -c digit-recognizer -m "maxpool" -f submission.csv

100% 208k/208k [00:00<00:00, 803kB/s]
Successfully submitted to Digit Recognizer